In [ ]:
import os

In [ ]:
%pwd

'r:\\ML PROJECTS\\MODELS\\TEXT-summarizer-project\\research'

In [ ]:
os.chdir("../..")

In [ ]:
%pwd

'r:\\ML PROJECTS\\MODELS'

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [ ]:
from MyProject.constants import *
from MyProject.utils.common import read_yaml, create_directories

In [ ]:
class ConfigManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config['data_ingestion']


        create_directories([config.root_dir])



        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config['root_dir']),
            source_URL=config['source_URL'],
            local_data_file=Path(config['local_data_file']),
            unzip_dir=Path(config['unzip_dir'])
        )
        return data_ingestion_config

In [ ]:
import os
import urllib.request as request
import zipfile
from MyProject.logging import logger
from MyProject.utils.common import get_size 

In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_data(self) -> Path:
        logger.info("Starting data download...")
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                self.config.source_URL, 
                self.config.local_data_file
            )
            logger.info(f"Data downloaded successfully and saved to {filename} with size {get_size(Path(filename))}")
        else:
            logger.info("Data file already exists. Skipping download.")
        return self.config.local_data_file
    
    def extract_zip_file(self, zip_file_path: Path):
        logger.info("Starting data extraction...")
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(self.config.unzip_dir)
        logger.info(f"Data extracted successfully to {self.config.unzip_dir}")

In [ ]:
try:
    config_manager = ConfigManager()
    data_ingestion_config = config_manager.get_data_ingestion_config()
    
    data_ingestion = DataIngestion(config=data_ingestion_config)
    
    downloaded_file=data_ingestion.download_data()
    data_ingestion.extract_zip_file(downloaded_file)
except Exception as e:
    raise e

[2025-10-27 17:18:08,414 : INFO : common: YAML file R:\ML PROJECTS\MODELS\TEXT-summarizer-project\config\config.yaml loaded successfully]
[2025-10-27 17:18:08,421 : INFO : common: YAML file R:\ML PROJECTS\MODELS\TEXT-summarizer-project\params.yaml loaded successfully]
[2025-10-27 17:18:08,424 : INFO : common: Created directory at: artifacts]
[2025-10-27 17:18:08,427 : INFO : common: Created directory at: artifacts\data_ingestion]
[2025-10-27 17:18:08,429 : INFO : 870230852: Starting data download...]
[2025-10-27 17:18:08,431 : INFO : 870230852: Data file already exists. Skipping download.]
[2025-10-27 17:18:08,432 : INFO : 870230852: Starting data extraction...]
[2025-10-27 17:18:08,748 : INFO : 870230852: Data extracted successfully to artifacts\data_ingestion\extracted_data]
